<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/10_Transfer_Learning_Fine_Tuning_TF_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub

# Device setting

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

#Model Training

## Load data

In [ ]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

## Normalize

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

## Model

In [ ]:
def my_model():
  inputs = keras.Input(shape=(28, 28, 1))
  x = layers.Conv2D(32, 3, padding='same')(inputs)
  x = layers.Conv2D(64, 3, padding='same')(x)
  x = layers.MaxPooling2D()(x)

  x = layers.Conv2D(128, 3, padding='same')(x)
  x = layers.Conv2D(256, 3, padding='same')(x)
  x = layers.MaxPooling2D()(x)

  x = layers.Flatten()(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  
  return model

model = my_model()

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy'],
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)
model.summary()


Epoch 1/2
1875/1875 - 25s - loss: 0.1371 - accuracy: 0.9581 - 25s/epoch - 13ms/step
Epoch 2/2
1875/1875 - 23s - loss: 0.0779 - accuracy: 0.9774 - 23s/epoch - 12ms/step
313/313 - 2s - loss: 0.0662 - accuracy: 0.9787 - 2s/epoch - 6ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)    

## Model Save

In [ ]:
model.save('pretrained/')

INFO:tensorflow:Assets written to: pretrained/assets


# Pretrained Model

## Load data

In [ ]:
(x_train,y_train), (x_test,y_test) = mnist.load_data()

## Normalize

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

## Load Model

In [ ]:
model = keras.models.load_model('pretrained/')

Layer Freeze

In [ ]:
model.trainable = False

# 반복문을 활용한 방법 - 예제의 경우 모든 Layer가 trainable == True
for layer in model.layers:
  assert layer.trainable == False
  layer.trainable = False

In [ ]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 256)       295168    
                                                             

In [ ]:
model.trainable = False # Layer를 Freeze하면 실행 시간이 빨라짐

base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
final_outputs = layers.Dense(10)(base_outputs)

new_model = keras.Model(inputs=base_inputs, outputs=final_outputs)

new_model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

new_model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)

Epoch 1/3
1875/1875 - 9s - loss: 0.0735 - accuracy: 0.9777 - 9s/epoch - 5ms/step
Epoch 2/3
1875/1875 - 9s - loss: 0.0370 - accuracy: 0.9880 - 9s/epoch - 5ms/step
Epoch 3/3
1875/1875 - 9s - loss: 0.0260 - accuracy: 0.9915 - 9s/epoch - 5ms/step


In [ ]:
model.trainable = True

base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
final_outputs = layers.Dense(10)(base_outputs)

new_model = keras.Model(inputs=base_inputs, outputs=final_outputs)

new_model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

new_model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)

Epoch 1/3
1875/1875 - 23s - loss: 0.0964 - accuracy: 0.9707 - 23s/epoch - 12ms/step
Epoch 2/3
1875/1875 - 23s - loss: 0.0660 - accuracy: 0.9808 - 23s/epoch - 12ms/step
Epoch 3/3
1875/1875 - 23s - loss: 0.0687 - accuracy: 0.9816 - 23s/epoch - 12ms/step


# Pretrained Keras Model

In [ ]:
x = tf.random.normal(shape=(5, 299, 299, 3))
y = tf.constant([0, 1, 2, 3, 4])

model = keras.applications.InceptionV3(include_top=True) # include_top -> 가장 상단의 fully connected계층들을 포함 시킬지의 여부
incept_base_inputs = model.layers[0].input
incept_base_outputs = model.layers[-2].output
incept_final_outputs = layers.Dense(5)(incept_base_outputs)
incept_model = keras.Model(inputs=incept_base_inputs, outputs=incept_base_outputs)

incept_model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)
incept_model.fit(x, y, batch_size=1, epochs=15, verbose=2)



Epoch 1/15
5/5 - 11s - loss: 7.6135 - accuracy: 0.0000e+00 - 11s/epoch - 2s/step
Epoch 2/15
5/5 - 1s - loss: 7.4693 - accuracy: 0.0000e+00 - 510ms/epoch - 102ms/step
Epoch 3/15
5/5 - 1s - loss: 7.3729 - accuracy: 0.0000e+00 - 501ms/epoch - 100ms/step
Epoch 4/15
5/5 - 1s - loss: 7.3206 - accuracy: 0.4000 - 511ms/epoch - 102ms/step
Epoch 5/15
5/5 - 1s - loss: 7.3077 - accuracy: 0.0000e+00 - 500ms/epoch - 100ms/step
Epoch 6/15
5/5 - 1s - loss: 7.2812 - accuracy: 0.6000 - 510ms/epoch - 102ms/step
Epoch 7/15
5/5 - 1s - loss: 7.2611 - accuracy: 0.6000 - 502ms/epoch - 100ms/step
Epoch 8/15
5/5 - 1s - loss: 7.2550 - accuracy: 0.8000 - 505ms/epoch - 101ms/step
Epoch 9/15
5/5 - 1s - loss: 7.2439 - accuracy: 1.0000 - 503ms/epoch - 101ms/step
Epoch 10/15
5/5 - 1s - loss: 7.2382 - accuracy: 1.0000 - 509ms/epoch - 102ms/step
Epoch 11/15
5/5 - 1s - loss: 7.2349 - accuracy: 0.8000 - 506ms/epoch - 101ms/step
Epoch 12/15
5/5 - 1s - loss: 7.2321 - accuracy: 0.8000 - 513ms/epoch - 103ms/step
Epoch 13/15
5

# Pretrained Hub Model

In [ ]:
x = tf.random.normal(shape=(5, 299, 299, 3))
y = tf.constant([0, 1, 2, 3, 4])
url = 'https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5'

In [ ]:
base_model = hub.KerasLayer(url,input_shape=(299, 299, 3))
base_model.trainable = False

model = keras.Sequential([
      base_model, # FC가 포함되지 않은 모델
      # 아래로 원하는 Layer를 추가할 수 있음
      layers.Dense(128, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(5), # FC layer
])

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [ ]:
model.fit(x, y, batch_size=1, epochs=15, verbose=2)

Epoch 1/15
5/5 - 6s - loss: 2.4498 - accuracy: 0.0000e+00 - 6s/epoch - 1s/step
Epoch 2/15
5/5 - 0s - loss: 1.7340 - accuracy: 0.0000e+00 - 158ms/epoch - 32ms/step
Epoch 3/15
5/5 - 0s - loss: 1.4988 - accuracy: 0.4000 - 158ms/epoch - 32ms/step
Epoch 4/15
5/5 - 0s - loss: 1.3509 - accuracy: 0.6000 - 160ms/epoch - 32ms/step
Epoch 5/15
5/5 - 0s - loss: 1.2088 - accuracy: 1.0000 - 158ms/epoch - 32ms/step
Epoch 6/15
5/5 - 0s - loss: 1.0943 - accuracy: 1.0000 - 159ms/epoch - 32ms/step
Epoch 7/15
5/5 - 0s - loss: 0.9775 - accuracy: 1.0000 - 158ms/epoch - 32ms/step
Epoch 8/15
5/5 - 0s - loss: 0.7636 - accuracy: 1.0000 - 158ms/epoch - 32ms/step
Epoch 9/15
5/5 - 0s - loss: 0.7174 - accuracy: 1.0000 - 158ms/epoch - 32ms/step
Epoch 10/15
5/5 - 0s - loss: 0.5346 - accuracy: 1.0000 - 160ms/epoch - 32ms/step
Epoch 11/15
5/5 - 0s - loss: 0.4154 - accuracy: 1.0000 - 157ms/epoch - 31ms/step
Epoch 12/15
5/5 - 0s - loss: 0.3702 - accuracy: 1.0000 - 159ms/epoch - 32ms/step
Epoch 13/15
5/5 - 0s - loss: 0.268